For each frame in the video is a node. Between every two nodes, there is a cost associated with
adding and removing apples. This cost is equal to the size of the exclusive or of the points in the two frames.
after calculating the cost between every two nodes, we can use the travelling salesman heuristic to find a path
through the frames that minimizes the number of apples added and removed.

In [1]:
import cv2
import numpy as np
from pathlib import Path
# add current directory to path
import sys
sys.path.append("~/src/badapple/badapple-irl/")
from python_tsp2.heuristics import solve_tsp_simulated_annealing, solve_tsp_lin_kernighan
from python_tsp2.exact import solve_tsp_dynamic_programming

In [2]:
# setup
badapple_path = Path(".").resolve().parent / "badapple-small.mp4"
square_side = 30
# skip to frame 364 because I already did every frame before that
frame_start = 365

In [3]:
# get all the points in each frame
cap = cv2.VideoCapture(str(badapple_path))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"width: {width}, height: {height}, fps: {fps}, n_frames: {n_frames}")
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_start)

frame_count = 0
frame_points = []
original_frame_id_map = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if frame_count % 100 == 0:
        print(f"frame {frame_count}/{(n_frames-frame_start)//2}", end="\r")
    frame_count += 1

    current_frame_pixels = set()
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            section = frame[j: j + square_side, i: i + square_side]
            if (np.mean(section)) < 50:
                current_frame_pixels.add((i, j))
    frame_points.append(current_frame_pixels)
    original_frame_id_map[len(frame_points) - 1] = frame_start + (frame_count-1) * 2  # trust
    # skip a frame because I don't want to do every frame
    ret, frame = cap.read()
num_frames = len(frame_points)
print(f"\nnum_frames: {num_frames}")

width: 960, height: 720, fps: 30, n_frames: 6572
frame 3100/3103
num_frames: 3104


In [4]:
# calculate the cost between every two frames (distance matrix), and then run TSP algorithm to get optimized order of frames
costs = np.zeros((num_frames, num_frames))
for i in range(num_frames):
    for j in range(num_frames):
        costs[i][j] = len(frame_points[i] ^ frame_points[j])
        costs[j][i] = costs[i][j]

In [15]:
import sys
import time
print(costs)
sys.setrecursionlimit(100000) 
start_time = time.time()
permutation, distance = solve_tsp_lin_kernighan(costs, x0=list(range(num_frames)), verbose=True)
print(permutation)
print(f"Time taken: {time.time() - start_time}")
print("Distance: ", distance)


[[  0.  13.  32. ... 759. 759. 759.]
 [ 13.   0.  23. ... 746. 746. 746.]
 [ 32.  23.   0. ... 739. 739. 739.]
 ...
 [759. 746. 739. ...   0.   0.   0.]
 [759. 746. 739. ...   0.   0.   0.]
 [759. 746. 739. ...   0.   0.   0.]]
Current value: 128084.0; Ejection chain: 1
Current value: 128080.0; Ejection chain: 1
Current value: 128044.0; Ejection chain: 1
Current value: 127928.0; Ejection chain: 1
Current value: 127802.0; Ejection chain: 1
Current value: 127732.0; Ejection chain: 1
Current value: 127730.0; Ejection chain: 1
Current value: 127722.0; Ejection chain: 1
Current value: 127708.0; Ejection chain: 1
Current value: 127708.0; Ejection chain: 1
Current value: 127706.0; Ejection chain: 1
Current value: 127696.0; Ejection chain: 1
Current value: 127682.0; Ejection chain: 1
Current value: 127680.0; Ejection chain: 1
Current value: 127678.0; Ejection chain: 1
Current value: 127678.0; Ejection chain: 1
Current value: 127618.0; Ejection chain: 1
Current value: 127600.0; Ejection chain: 

In [19]:
# calculate the number of apple moves using the normal order and optimized order of frames
base_apple_moves = 0
for i in range(num_frames - 1):
    base_apple_moves += len(frame_points[i] ^ frame_points[i + 1])
print(f"Base apple moves: {base_apple_moves}")
optimized_apple_moves = 0
for i in range(num_frames - 1):
    optimized_apple_moves += len(frame_points[permutation[i]] ^ frame_points[permutation[i + 1]])
print(f"Optimized apple moves: {optimized_apple_moves}, percentage of original: {optimized_apple_moves / base_apple_moves * 100}%")

Base apple moves: 127325
Optimized apple moves: 120226, percentage of original: 94.42450422148046%


In [17]:
# generate new diff frames using the optimized order of frames
circle_frames_dir = Path("circle_frames_opt")
circle_frames_dir.mkdir(exist_ok=True)
diff_frames_dir = Path("diff_frames_opt")
diff_frames_dir.mkdir(exist_ok=True)
# actually load permutation from permutation.txt because the above is non-deterministic
with open("permutation.txt", "r") as f:
    permutation = [int(i) for i in f.read().split()]
for index in range(num_frames):
    frame1 = np.ones((height, width, 3), dtype=np.uint8) * 255
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            if (i, j) in frame_points[permutation[index]]:
                text_color = (255, 255, 255)
                cv2.circle(frame1, (i+square_side//2, j+square_side//2), square_side//2, 0, -1)
            else:
                text_color = 0
            cv2.putText(frame1, f"{j//square_side+1}", (i+square_side//3-1, j + 22), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
            cv2.putText(frame1, f"{i//square_side+1}", (i+square_side//3-1, j + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
    cv2.imwrite(str(circle_frames_dir / f"{index}.png"), frame1)
   

In [20]:
# print permutation to a file
with open("permutation.txt", "w") as f:
    for i in permutation:
        f.write(f"{original_frame_id_map[i]}\n")
print(len(permutation))

3104


In [18]:
# generate diff frames by comparing current frame with previous frame's set, making green for additions and red for removals

last_frame_pixels = set()
for index in range(num_frames-1):
        
    out_frame = np.ones((height, width, 3), dtype=np.uint8) * 255
    current_frame_pixels = frame_points[permutation[index]]
    
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            if (i, j) in current_frame_pixels and (i, j) in last_frame_pixels:
                # persisted from previous frame. Black circle, white text
                circle_colour = (0, 0, 0)
                text_color = (255, 255, 255)
            elif (i, j) in current_frame_pixels:
                # new black dot. Green circle, black text
                circle_colour = (0, 255, 0)
                text_color = (0, 0 ,0)
            elif (i, j) in last_frame_pixels:
                # removed black dot. Red circle, black text
                circle_colour = (0, 0, 255)
                text_color = (0, 0, 0)
            else:
                # persisted nothing. Just use black text
                text_color = (0, 0 ,0)
                circle_colour = None
            if circle_colour is not None:
                cv2.circle(out_frame, (i+square_side//2, j+square_side//2), square_side//2, circle_colour, -1)
            cv2.putText(out_frame, f"{j//square_side+1}", (i+square_side//3-1, j + 22), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
            cv2.putText(out_frame, f"{i//square_side+1}", (i+square_side//3-1, j + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)

    cv2.imwrite(f"diff_frames_opt/{index}.png", out_frame)

    last_frame_pixels = current_frame_pixels

In [22]:
hashset = set()
hashmap = {}
duplicates = {}
total_duplicates = 0
flat_duplicates_list = []
for index, current_frame_pixels in enumerate([frame_points[i] for i in permutation]):
    hash_of_frame = hash(frozenset(current_frame_pixels))
    if hash_of_frame in hashset:
        original_frame = hashmap[hash_of_frame]
        print(f"Frame {index} is a duplicate of {original_frame}")
        duplicates[original_frame].append(index)
        total_duplicates += 1
        flat_duplicates_list.append(index)
    else:
        hashset.add(hash_of_frame)
        hashmap[hash_of_frame] = index
        duplicates[index] = []

print("All duplicates:")
print(duplicates)
print(f"Total duplicates: {total_duplicates}")
print(f"Flat duplicates list: {flat_duplicates_list}")

Frame 97 is a duplicate of 96
Frame 98 is a duplicate of 96
Frame 103 is a duplicate of 102
Frame 242 is a duplicate of 241
Frame 243 is a duplicate of 241
Frame 244 is a duplicate of 241
Frame 245 is a duplicate of 241
Frame 246 is a duplicate of 241
Frame 247 is a duplicate of 241
Frame 248 is a duplicate of 241
Frame 249 is a duplicate of 241
Frame 250 is a duplicate of 241
Frame 251 is a duplicate of 241
Frame 252 is a duplicate of 241
Frame 253 is a duplicate of 241
Frame 254 is a duplicate of 241
Frame 255 is a duplicate of 241
Frame 256 is a duplicate of 241
Frame 257 is a duplicate of 241
Frame 258 is a duplicate of 241
Frame 259 is a duplicate of 241
Frame 260 is a duplicate of 241
Frame 261 is a duplicate of 241
Frame 262 is a duplicate of 241
Frame 263 is a duplicate of 241
Frame 264 is a duplicate of 241
Frame 265 is a duplicate of 241
Frame 266 is a duplicate of 241
Frame 267 is a duplicate of 241
Frame 268 is a duplicate of 241
Frame 269 is a duplicate of 241
Frame 270 is